In [1]:
library(igraph)
library(dplyr)
library(ggraph)


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union



Attaching package: ‘dplyr’


The following objects are masked from ‘package:igraph’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggplot2



In [2]:
modules <- read.csv("modules_micro3_AD.csv")
central_gene <- "PSEN1"

In [3]:
# Find the module associated with the central gene
central_module <- modules %>%
  filter(gene_name == central_gene) %>%
  pull(module)

In [4]:
# Get all genes in the same module
module_genes <- modules %>%
  filter(module == central_module)

In [6]:
# Limit to top 20 genes by kME score (include PSEN1)
top_genes <- module_genes %>%
  arrange(desc(kME_micro_state3_AD3)) %>%
  slice_head(n = 20)  # Adjust n as needed

In [7]:
# Ensure PSEN1 is included in the top genes
if (!central_gene %in% top_genes$gene_name) {
  top_genes <- bind_rows(top_genes, module_genes %>% filter(gene_name == central_gene))
}

In [8]:
# Filter edge list to include only top genes
edge_list <- expand.grid(
  from = top_genes$gene_name,
  to = top_genes$gene_name
) %>%
  filter(from != to) %>%  # Exclude self-loops
  mutate(weight = (top_genes$kME_micro_state3_AD3[match(from, top_genes$gene_name)] +
                   top_genes$kME_micro_state3_AD3[match(to, top_genes$gene_name)]) / 2)  # Average kME as weight

In [9]:
# Ensure PSEN1 connections are retained
filtered_edges_psen1 <- edge_list %>%
  filter(from == "PSEN1" | to == "PSEN1")

In [10]:
# Combine PSEN1 edges and other edges above a threshold
threshold <- 0.7  # Adjust the threshold as needed
filtered_edges_subnetwork <- edge_list %>%
  filter(weight > threshold)

final_filtered_edges <- bind_rows(filtered_edges_psen1, filtered_edges_subnetwork)

In [11]:
# Create graph object using the filtered edge list
network <- graph_from_data_frame(final_filtered_edges, directed = FALSE)

In [12]:
# Save plot as a PDF
pdf("PSEN1_network_plot_mic3_AD.pdf", width = 10, height = 10)  # Specify file name and dimensions
ggraph(network, layout = "fr") +  #Force-directed layout
  geom_edge_link(aes(alpha = weight), color = "blue") +  # Scale thickness by weight
  geom_node_point(aes(filter = name == central_gene), size = 7, color = "red") +  # Highlight central gene
  geom_node_point(aes(filter = name != central_gene), size = 3, color = "gray") +
  geom_node_text(aes(label = name), repel = TRUE, size = 4, color = "black", max.overlaps = 50) +
  theme_void() +
  ggtitle(paste("Reduced Network of cellstate 3" ,central_module, "Centered on", central_gene)) +
  theme(plot.margin = margin(30, 30, 30, 30))
dev.off() 

png 
  2